## FLORES Evaluation Model Runner

This script runs the FLORES flood simulation numerous times, changing the input variables every time. The resulting file can be analyzed in the 'FLORES - Evaluation Model Analysis' script.

The FLORES evaluation largely depends on the 'Exploratory Modelling and Analysis (EMA) Workbench', built and maintained by J.H. Kwakkel.

Other used python-packages are:
- mpld3
- ipyparallel
- pydotplus
- seaborn
- Graphviz
- SALib
- Platypus
- Borg MOEA

Version FLORES flood simulation: V1.1

Version EMA-Workbench: V1.2.1 (26-08-2018)

Last Updated: 21-08-2019

## LOG
The following evaluations have been run:
    
Date   ...     Evaluation     ...                              Storm  ...    Nr. strategies ...      Name

<p>--/--/---- ... description   ...    # ...          #      ...             /name</p>



In [1]:
import sys
sys.path.append('./Packages')
%pwd

'D:\\ecvanberchum\\Surfdrive\\MODOS\\MODOS_model\\FLORES_main'

In [4]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)



from ema_workbench.em_framework import (RealParameter, ScalarOutcome, 
                           perform_experiments, CategoricalParameter, samplers, Scenario)

from ema_workbench.util import (ema_logging)
from ema_workbench import (Model, RealParameter, ScalarOutcome, Constant,
                           ema_logging, MultiprocessingEvaluator)
from ema_workbench.em_framework.model import (Replicator,BaseModel)
from ema_workbench.em_framework.samplers import (MonteCarloSampler, FullFactorialSampler, LHSSampler,
                       PartialFactorialSampler, sample_levers, sample_uncertainties)
from ema_workbench.util import (save_results, load_results)

from Library.simulation_calculations_beira import run_hydraulic_calculation
from Library.simulation_definitions_beira import (Impact)

import Library.simulation_data_beira as data

%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as ss
import numpy as np
import seaborn as sns
from timeit import default_timer as timer
import csv
from scipy.integrate import trapz

time_prep = timer()
basins_master = data.load_basins("Library/input_data/region_layout_basins.csv",
                            "Library/input_data/urban_development_scenarios.csv")  # Regional Layout
layers_master = data.load_layers("Library/input_data/region_layout_layers.csv", basins_master)
all_measures_master = data.load_measures("Library/input_data/flood_risk_reduction_measures.csv")
hydraulic_conditions_master = data.load_hydraulic_conditions("Library/input_data/hydraulic_boundary_conditions_surge_beira.csv",
                                                        "Library/input_data/hydraulic_boundary_conditions_rain_beira.csv")

damage_curves = data.load_damage_curves('Library/input_data/global_flood_depth-damage_functions__30102017.xlsx', 'AFRICA',
                                   'Mozambique', 'Object based', 1.30)
data.load_basin_borders(basins_master, "Library/input_data/region_layout_basin_borders.csv")
drainage_master = data.load_drainage_capacities(basins_master, 'Library/input_data/basins_drainage.csv', small_channel=6,
                                           mid_channel=10, large_channel=35, low_drain=0, mid_drain=2, high_drain=4)


In [5]:
def flood_simulation_model(return_period_storm = 0,    # 0, 2, 5, 10, 50, 100
                             return_period_rain = 0,    # 0, 2, 5, 10, 50, 100
                             struc_measure_coast_1 = 'none',  # 'Heighten dunes east','Sand supplements east', None
                             struc_measure_coast_2 = 'none',  # 'Heighten dunes west','Floodwall west'', None
                             struc_measure_inland_1 = 'none',  # 'Heighten inland road', None
                             struc_measure_inland_2 = 'none',
                             drainage_measure_1 = 'none',  # 'Second phase drainage', None
                             drainage_measure_2 = 'none',   # 'Microdrainage, None
                             retention_measure_1 = 'none',  # 'East retention', None
                             retention_measure_2 = 'none',    # 'Maraza retention, None
                             emergency_measure_1 = 'none',  # 'Improve evacuation', None
                             emergency_measure_2 = 'none',    # 'Early warning system', None
                             h_struc_measure_coast_1 = 0,  # 8-12
                             h_struc_measure_coast_2 = 0,  # 8-12
                             h_struc_measure_inland_1 = 0,  # 7-10
                             h_struc_measure_inland_2 = 0,
                             input_scenario_climate = 'none',  # 'high','low', 'none'
                             input_scenario_development = 'none'  # 'high','low', 'none'
                            ):
        # added so we can add scenario information into the replicator
    if return_period_rain == 'INFO':
        scenario_info = str(input_scenario_climate) + ',' + str(input_scenario_development)
        return [scenario_info]*3
    
        # start of model, loads simulation-specific data
    hydraulic = data.get_hydraulic_boundary_conditions(hydraulic_conditions_master, return_period_storm, return_period_rain,
                                                  input_scenario_climate)
    region_layout = data.get_region_layout(basins_master, layers_master, input_scenario_development)
    strategy = data.get_strategy(all_measures_master, region_layout,
                            [struc_measure_coast_1, struc_measure_coast_2, struc_measure_inland_1, struc_measure_inland_2,
                             drainage_measure_1, drainage_measure_2, retention_measure_1, retention_measure_2,
                             emergency_measure_1, emergency_measure_2],
                            [h_struc_measure_coast_1, h_struc_measure_coast_2, h_struc_measure_inland_1,
                             h_struc_measure_inland_2])
    impact = Impact()

    # Builds and correctly names the scenarios
    for sequence in [1, 3]:
        region_layout.Layers[sequence].get_scenarios()
    strategy.get_list_scenarios(region_layout)

    # Hydraulic calculations, runs entire hydraulic simulation (pluvial and storm surge flooding)
    run_hydraulic_calculation(region_layout, hydraulic, strategy)

    strategy.get_probabilities(region_layout, hydraulic)

    # Calculate cost of construction and repair
    total_cost = strategy.get_construction_costs()

    # Impact calculations, calculates expected damages and exposed population per basin and in total
    impact.run_impact_calculations(region_layout, strategy, damage_curves)
    risk_reduction = impact.TotalExpectedDamage
    construction_cost = total_cost
    affected_pop_reduction = impact.TotalExpectedExposedPop
    
    return risk_reduction, construction_cost, affected_pop_reduction

In [6]:
'''
Created on

.. codeauthor:: jhkwakkel <j.h.kwakkel (at) tudelft (dot) nl>
'''
class ReplicatorModel(Replicator, BaseModel):
    pass

def process_risk(data_risk):
    D0_source = {'low,low': 9800000,  
                 'low,high': 24092900, # not calibrated. don't use
                 'high,low': 14980000,  
                 'high,high': 31478200  # not calibrated. don't use
                 }

    scenario_info = data_risk.pop()
    D0 = D0_source[scenario_info]  # Base case, storm: 0 year
    data_risk.append(0)  # running the simulation without storm or rain is skipped
    Prob_rain = [0, 0.01, 0.02, 0.1, 0.2, 1]
    Prob_storm = [0, 0.01, 0.02, 0.1, 0.2, 1]
    runs_per_hazard = len(Prob_rain) - 1
    data_risk_array = np.reshape(data_risk, (runs_per_hazard, runs_per_hazard))

    risk_conditional_storm = []
    for count_storm, p_storm in enumerate(Prob_storm[0:5]):
        tmp_conditional_damages = data_risk_array[count_storm]
        conditional_damages = np.append(tmp_conditional_damages, tmp_conditional_damages[-1])
        risk_conditional_storm.append(trapz(conditional_damages, Prob_rain))
    risk_conditional_storm.append(risk_conditional_storm[-1])
    risk = trapz(risk_conditional_storm, Prob_storm)
    risk_reduction = (D0 - risk) / D0

    return risk_reduction
    
def process_affected_people(data_people):

    P0_source = {'low,low': 32800, 
                 'low,high': 80100,  #not calibrated. don't use
                 'high,low': 48700, 
                 'high,high': 112100  #not calibrated. don't use
                 }

    scenario_info = data_people.pop()
    P0 = P0_source[scenario_info]
    data_people.append(0)

    Prob_rain = [0, 0.01, 0.02, 0.1, 0.2, 1]
    Prob_storm = [0, 0.01, 0.02, 0.1, 0.2, 1]
    runs_per_hazard = len(Prob_rain) - 1
    data_people_array = np.reshape(data_people, (runs_per_hazard, runs_per_hazard))

    people_conditional_storm = []
    for count_storm, p_storm in enumerate(Prob_storm[0:5]):
        tmp_conditional_people = data_people_array[count_storm]
        conditional_people = np.append(tmp_conditional_people, tmp_conditional_people[-1])
        people_conditional_storm.append(trapz(conditional_people, Prob_rain))

    people_conditional_storm.append(people_conditional_storm[-1])
    affected_population = trapz(people_conditional_storm, Prob_storm)
    affected_population_reduction = (P0 - affected_population) / P0

    
    return affected_population_reduction

def pick_one(data):
    return data[0]
     
def pick_50(data):
    return data[2]

if __name__ == '__main__':
    ema_logging.log_to_stderr(level=ema_logging.INFO)
    model = ReplicatorModel('EvaluationModel', function=flood_simulation_model)
       
    model.replications = [{"return_period_rain":100,"return_period_storm":100},
                      {"return_period_rain":50,"return_period_storm":100},
                      {"return_period_rain":10,"return_period_storm":100},
                      {"return_period_rain":5,"return_period_storm":100},
                      {"return_period_rain":0,"return_period_storm":100}, 
                      {"return_period_rain":100,"return_period_storm":50},
                      {"return_period_rain":50,"return_period_storm":50},
                      {"return_period_rain":10,"return_period_storm":50},
                      {"return_period_rain":5,"return_period_storm":50},
                      {"return_period_rain":0,"return_period_storm":50},
                      {"return_period_rain":100,"return_period_storm":10},
                      {"return_period_rain":50,"return_period_storm":10},
                      {"return_period_rain":10,"return_period_storm":10},
                      {"return_period_rain":5,"return_period_storm":10},
                      {"return_period_rain":0,"return_period_storm":10},
                      {"return_period_rain":100,"return_period_storm":5},
                      {"return_period_rain":50,"return_period_storm":5},
                      {"return_period_rain":10,"return_period_storm":5},
                      {"return_period_rain":5,"return_period_storm":5},
                      {"return_period_rain":0,"return_period_storm":5},
                      {"return_period_rain":100,"return_period_storm":0},
                      {"return_period_rain":50,"return_period_storm":0},
                      {"return_period_rain":10,"return_period_storm":0},
                      {"return_period_rain":5,"return_period_storm":0},
                      {"return_period_rain":'INFO',"return_period_storm":'INFO'}]



    # set levers
    model.levers = [CategoricalParameter('struc_measure_coast_1',['Heighten_dunes_east','Sand_supplements_east','none']),
                    CategoricalParameter('struc_measure_coast_2',['Heighten_dunes_west','Floodwall_west', 'none']),
                    CategoricalParameter('struc_measure_inland_1',['Heighten_inland_road', 'none']),
                    CategoricalParameter('drainage_measure_1',['Second_phase_drainage','none']),
                    CategoricalParameter('drainage_measure_2',['Microdrainage', 'none']),
                    CategoricalParameter('retention_measure_1',['East_retention', 'none']),
                    CategoricalParameter('retention_measure_2',['Chota_retention', 'none']),
                    CategoricalParameter('emergency_measure_1',['Improve_evacuation', 'none']),
                    CategoricalParameter('emergency_measure_2',['Early_warning_system', 'none']),                   
                    RealParameter("h_struc_measure_coast_1", 9, 12),
                    RealParameter("h_struc_measure_coast_2", 8, 12),
                    RealParameter("h_struc_measure_inland_1", 7.5, 9)                    
                    ]
    
    # set constants 
    model.constants = [Constant('input_scenario_development','low'),
                       ]
    
    # set uncertainties, future scenarios
    model.uncertainties = [CategoricalParameter('input_scenario_climate',['low','high']),
                          ]
    
    #specification of the outcomes
    model.outcomes = [ScalarOutcome("risk_reduction", function=process_risk),
                      ScalarOutcome("construction_costs", function=pick_one),
                      ScalarOutcome("affected_pop_reduction", function=process_affected_people)]
    

    nr_strategies = 10
    
    fn = './data/{} experiments_FLORES_Beira_21-08-2019-2_scenarios.tar.gz'.format(nr_strategies)
print('ready')

ready


In [7]:
# Single processor runner

start = timer()
try:
    # check whether it overwrites another file
    results = load_results(fn)
except FileNotFoundError:
    # generate some random policies by sampling over levers
    system_configurations = samplers.sample_levers(model, nr_strategies)
    
    results = perform_experiments(model, scenarios=4, policies=nr_strategies, 
                                  uncertainty_sampling='ff',reporting_interval=1) 
    save_results(results, fn)

print("done")
end = timer()
print(end - start)

[MainProcess/INFO] performing 2 scenarios * 10 policies * 1 model(s) = 20 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 6 cases completed
[MainProcess/INFO] 7 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 9 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 11 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 13 cases completed
[MainProcess/INFO] 14 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 17 cases completed
[MainProcess/INFO] 18 cases completed
[MainProcess/INFO] 19 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] results saved successfully to D:\ecvanberchum\Sur

done
4802.800773702332


In [ ]:
#Multi processor runner

from ema_workbench.em_framework.evaluators import (MultiprocessingEvaluator)

start = timer()
try:
    results = load_results(fn)
except FileNotFoundError:
    system_configurations = samplers.sample_levers(model, nr_strategies)    
    
    with MultiprocessingEvaluator(model) as evaluator:
        evaluator.n_processes = 2
        evaluator.perform_experiments(scenarios=4, policies=nr_strategies, 
                                  uncertainty_sampling='ff',reporting_interval=1)
        print("here")
    save_results(results, fn)

print("done")
end = timer()
print(end - start)

In [11]:
import matplotlib
matplotlib.__version__

'2.2.2'